In [57]:
# Data Reference Classifier for Make Data Count Competition

import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [ ]:
# Download stopwords
nltk.download('stopwords')

# === Step 1: Load Dataset ===
data_dir = Path("D:/Jupyter File/DML_Kelompok-B/Dataset")
train_labels = pd.read_csv(data_dir / "train_labels.csv")
sample_submission = pd.read_csv(data_dir / "sample_submission.csv")
train_xml_dir = data_dir / "train/XML"
test_xml_dir = data_dir / "test/XML"

[nltk_data] Downloading package stopwords to C:\Users\Nugroho
[nltk_data]     Agung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'sample_submission.csv'

In [ ]:
# Ekstrak teks dari XML
def extract_text_from_xml(xml_path):
    with open(xml_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'xml')
    body = soup.find('body')
    return body.get_text(separator=' ', strip=True) if body else ""

In [ ]:
# Preprocessing teks dasar
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', ' ', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

In [ ]:
# Bangun dataset berdasarkan article_id dan label type
samples = []
for _, row in train_labels.iterrows():
    article_id = row['article_id']
    label = row['type'].lower()
    if label not in ['primary', 'secondary']:
        continue  # abaikan Missing
    xml_file = train_xml_dir / f"{article_id}.xml"
    if not xml_file.exists():
        continue
    text = extract_text_from_xml(xml_file)
    text = preprocess(text)
    samples.append((text, label))

train_df = pd.DataFrame(samples, columns=['text', 'label'])

(0, 2)
Empty DataFrame
Columns: [text, label]
Index: []


In [ ]:
# Train-test split dan pipeline TFIDF + Logistic Regression
X = train_df['text']
y = train_df['label']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

joblib.dump(model, data_dir / "model.joblib")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# === Step 6: Inference on Test Set ===
test_predictions = []
model = joblib.load("model.joblib")

for xml_file in test_xml_dir.glob("*.xml"):
    doc_id = xml_file.stem
    full_text = extract_text_from_xml(xml_file)
    # Dummy chunking: sliding window (can be improved)
    chunk_size = 30
    stride = 15
    predictions = []
    for i in range(0, len(full_text) - chunk_size, stride):
        chunk = full_text[i:i+chunk_size]
        preprocessed = preprocess(chunk)
        if not preprocessed.strip():
            continue
        pred = model.predict([preprocessed])[0]
        predictions.append(f"{pred} {i} {i+chunk_size}")
    prediction_string = "|".join(predictions)
    test_predictions.append((doc_id, prediction_string))

submission_df = pd.DataFrame(test_predictions, columns=['Id', 'PredictionString'])
submission_df.to_csv("submission.csv", index=False)
print("Submission saved as submission.csv")
